In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, timedelta, date

In [ ]:
main_df = pd.read_csv('../data/requests_census_neighborhoods.csv')

In [ ]:
main_df.columns = main_df.columns.str.strip()

In [ ]:
main_df['Households (Median Income)'] = main_df['Households (Median Income)'].str.replace(',', '')

In [ ]:
main_df['Households (Median Income)'] = main_df['Households (Median Income)'].astype(int)

In [ ]:
main_df['Income Bracket'] = ''

for ind, row in main_df.iterrows():
    if row['Households (Median Income)'] < 40000:
        main_df.loc[ind, 'Income Bracket'] = 'Less Than $40,000'
    elif row['Households (Median Income)'] >= 40000 and row['Households (Median Income)'] < 60000:
        main_df.loc[ind, 'Income Bracket'] = '$40,000 to $60,000'
    elif row['Households (Median Income)'] >= 60000 and row['Households (Median Income)'] < 80000:
        main_df.loc[ind, 'Income Bracket'] = '$60,000 to $80,000'
    elif row['Households (Median Income)'] >= 80000 and row['Households (Median Income)'] < 100000:
        main_df.loc[ind, 'Income Bracket'] = '$80,000 to $100,000'
    elif row['Households (Median Income)'] >= 100000:
        main_df.loc[ind, 'Income Bracket'] = 'More Than $100,000'
    else:
        main_df.loc[ind, 'Income Bracket'] = 'Other'

main_df.head()        

In [ ]:
main_df['Households (Count)'] = main_df['Households (Count)'].str.replace(',', '')
main_df['Households (Count)'].value_counts()
#main_df['Households (Count)'] = main_df['Households (Count)'].astype(int)

In [ ]:
main_df.info()

In [ ]:
main_df['Black or African American (Percent Distribution)'] = main_df['Black or African American (Percent Distribution)'].str.replace('%', '')
main_df['Black or African American (Percent Distribution)'] = main_df['Black or African American (Percent Distribution)'].astype(float)

In [45]:
main_df['American Indian and Alaska Native (Percent Distribution)'] = main_df['American Indian and Alaska Native (Percent Distribution)'].str.replace('%', '')
main_df['American Indian and Alaska Native (Percent Distribution)'] = main_df['American Indian and Alaska Native (Percent Distribution)'].astype(float)

In [46]:
main_df['Asian (Percent Distribution)'] = main_df['Asian (Percent Distribution)'].str.replace('%', '')
main_df['Asian (Percent Distribution)'] = main_df['Asian (Percent Distribution)'].astype(float)

In [47]:
main_df['Native Hawaiian and Other Pacific Islander (Percent Distribution)'] = main_df['Native Hawaiian and Other Pacific Islander (Percent Distribution)'].str.replace('%', '')
main_df['Native Hawaiian and Other Pacific Islander (Percent Distribution)'] = main_df['Native Hawaiian and Other Pacific Islander (Percent Distribution)'].astype(float)

In [48]:
main_df['Some other race (Percent Distribution)'] = main_df['Some other race (Percent Distribution)'].str.replace('%', '')
main_df['Some other race (Percent Distribution)'] = main_df['Some other race (Percent Distribution)'].astype(float)

In [55]:
main_df['Hispanic or Latino origin (of any race) (Percent Distribution)'] = main_df['Hispanic or Latino origin (of any race) (Percent Distribution)'].str.replace('%', '')
main_df['Hispanic or Latino origin (of any race) (Percent Distribution)'] = main_df['Hispanic or Latino origin (of any race) (Percent Distribution)'].astype(float)

In [59]:
main_df['White (Percent Distribution)'] = main_df['White (Percent Distribution)'].str.replace('%', '')
main_df['White (Percent Distribution)'] = main_df['White (Percent Distribution)'].astype(float)

In [61]:
main_df['Two or more races (Percent Distribution)'] = main_df['Two or more races (Percent Distribution)'].str.replace('%', '')
main_df['Two or more races (Percent Distribution)'] = main_df['Two or more races (Percent Distribution)'].astype(float)

In [62]:
main_df['White alone, not Hispanic or Latino (Percent Distribution)'] = main_df['White alone, not Hispanic or Latino (Percent Distribution)'].str.replace('%', '')
main_df['White alone, not Hispanic or Latino (Percent Distribution)'] = main_df['White alone, not Hispanic or Latino (Percent Distribution)'].astype(float)

In [65]:
main_df['White alone, not Hispanic or Latino (Percent Distribution)'].value_counts()

68.9    74870
41.4    68662
62.3    65905
26.0    50040
67.2    48384
36.9    40019
61.3    35983
47.7    35596
72.4    32954
91.1    30777
46.3    29445
89.4    27150
72.6    26786
51.9    25809
81.2    20749
78.2    20628
18.1    20541
85.5    12339
86.8    11753
85.6     8339
83.4     7515
83.6     7457
96.2     6579
73.1     5084
51.0     3506
86.4     2163
39.7     1292
85.7      484
52.6      396
91.9      345
96.1      247
84.6      229
Name: White alone, not Hispanic or Latino (Percent Distribution), dtype: int64

In [56]:
main_df['Minority Population %'] = main_df['Black or African American (Percent Distribution)'] + main_df['American Indian and Alaska Native (Percent Distribution)'] + main_df['Asian (Percent Distribution)'] + main_df['Native Hawaiian and Other Pacific Islander (Percent Distribution)'] + main_df['Some other race (Percent Distribution)'] + main_df['Hispanic or Latino origin (of any race) (Percent Distribution)']

In [68]:
main_df['Minority Population Percent'] = 100 - main_df['White alone, not Hispanic or Latino (Percent Distribution)']

In [ ]:
main_df['Minority Population %'].value_counts()

In [ ]:
main_df['Minority Population Percent'].value_counts()

In [71]:
del main_df['Minority Population %']

In [72]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722026 entries, 0 to 722025
Data columns (total 57 columns):
 #   Column                                                             Non-Null Count   Dtype  
---  ------                                                             --------------   -----  
 0   ZIP                                                                722026 non-null  int64  
 1   Households (Count)                                                 722026 non-null  object 
 2   White (Count)                                                      722026 non-null  object 
 3   Black or African American (Count)                                  722026 non-null  object 
 4   American Indian and Alaska Native (Count)                          722026 non-null  float64
 5   Asian (Count)                                                      722026 non-null  object 
 6   Native Hawaiian and Other Pacific Islander (Count)                 722026 non-null  float64
 7   Some other 

In [ ]:
# income_zips = main_df.iloc[:, [0, 21, 33, 34, 35, 53, 54]]
# income_zips.info()

In [ ]:
# def resolve_mean(income_zips):
#     return income_zips['Days to Resolve'].mean()
# income_zips_group = income_zips.groupby(['Neighborhood', 'Households (Median Income)']).apply(resolve_mean)
# income_zips_group = income_zips_group.reset_index()
# plt.figure(figsize = (16, 8))
# sns.scatterplot(data = income_zips_group, x = 'Households (Median Income)', y = 0)
# plt.ylabel('Avg Days to Resolve Request');

In [ ]:
# income_zips_group.sort_values(by = ['Households (Median Income)'], ascending = False)

In [ ]:
# main_df.groupby(['Neighborhood', 'Request Type']).agg({'Days to Resolve':'mean'})

In [ ]:
# main_df.groupby('Neighborhood')['Request Type'].value_counts()